In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
from notebooks.imports import *

In [2]:
from kilosort import run_kilosort, io
import spikeinterface.extractors as se

### Load Configs

In [9]:
from config import dir_config, main_config, s_probe_path

raw_dir = Path(dir_config.data.raw)
sorted_dir = Path(dir_config.data.sorted)

### Utils functions

In [40]:
def convert_nsx_to_bin(nsx_path, nsx_name, bin_path, bin_name):
    
    nsx_data, N, c, s, fs, probe_path = None, None, None, None, None, None
    
    nsx_file = Path(nsx_path, nsx_name)
    if nsx_file.exists():
        nsx_data = se.read_blackrock(file_path=Path(nsx_path, nsx_name), stream_id='5')   
        filename, N, c, s, fs, probe_path = io.spikeinterface_to_binary(
        nsx_data, bin_path, data_name=f"{bin_name}.bin", dtype=np.int16,
        chunksize=60000, export_probe=True, probe_name='probe.prb'
        )
        print(f"{N}, {c}, {s}, {fs}, {probe_path}")
    return nsx_data, N, c, s, fs, probe_path

## Covert blackrock (NSX) data to bin

In [41]:
# loop through all folders in raw data directory using Path
for session_path in Path(raw_dir).iterdir():
    # get folder path, folder name and bin path
    nxs_path = str(session_path)
    nsx_name = session_path.name
    bin_path = str(Path(sorted_dir, nsx_name))
    bin_name = session_path.name
    nsx_data, N, c, s, fs, probe_path = convert_nsx_to_bin(nsx_path=nxs_path, nsx_name=nsx_name, bin_path=bin_path, bin_name=bin_name)

In [39]:
nsx_data

## Run Kilosort

In [10]:
settings = {
    'fs': 30000,
    'batch_size': 120000,
    # 'nblocks': 1,
    'min_template_size': 30,
    'nearest_templates': 6,
}

probe = io.load_probe(s_probe_path)
assert probe is not None, 'No probe information exported by SpikeInterface'


In [20]:
drift_correction = True
if not drift_correction:
    settings['nblocks'] = 1
else:
    settings.pop('nblocks', None)


for session_path in Path(raw_dir).iterdir():
    try:
        bin_path = str(session_path)
        bin_name = session_path.name
        filename = f"{bin_path}/{bin_name}.bin"
        settings['n_chan_bin'] = 18

        ops, st, clu, tF, Wall, similar_templates, is_ref, est_contam_rate, kept_spikes = run_kilosort(
            settings=settings, probe=probe, filename=filename, progress_bar=True,
        )
    except Exception as e:
        print(f"Error in {bin_name}: {e}")


kilosort.run_kilosort: Kilosort version 4.0.13
kilosort.run_kilosort: Kilosort version 4.0.13
kilosort.run_kilosort: Kilosort version 4.0.13
kilosort.run_kilosort: Kilosort version 4.0.13
kilosort.run_kilosort: Kilosort version 4.0.13
kilosort.run_kilosort: Kilosort version 4.0.13
kilosort.run_kilosort: Kilosort version 4.0.13
kilosort.run_kilosort: Kilosort version 4.0.13
kilosort.run_kilosort: Kilosort version 4.0.13
kilosort.run_kilosort: Kilosort version 4.0.13
kilosort.run_kilosort: Sorting /mnt/prior-data/raw/210216_GP_JP/210216_GP_JP.bin
kilosort.run_kilosort: Sorting /mnt/prior-data/raw/210216_GP_JP/210216_GP_JP.bin
kilosort.run_kilosort: Sorting /mnt/prior-data/raw/210216_GP_JP/210216_GP_JP.bin
kilosort.run_kilosort: Sorting /mnt/prior-data/raw/210216_GP_JP/210216_GP_JP.bin
kilosort.run_kilosort: Sorting /mnt/prior-data/raw/210216_GP_JP/210216_GP_JP.bin
kilosort.run_kilosort: Sorting /mnt/prior-data/raw/210216_GP_JP/210216_GP_JP.bin
kilosort.run_kilosort: Sorting /mnt/prior-da

Error in 210216_GP_JP: Bytes in binary file did not divide evenly, incorrect n_chan_bin ('number of channels' in GUI).
Error in 210305_GP_JP: Bytes in binary file did not divide evenly, incorrect n_chan_bin ('number of channels' in GUI).


kilosort.run_kilosort: Using GPU for PyTorch computations. Specify `device` to change this.
kilosort.run_kilosort: Using GPU for PyTorch computations. Specify `device` to change this.
kilosort.run_kilosort: Using GPU for PyTorch computations. Specify `device` to change this.
kilosort.run_kilosort:  
kilosort.run_kilosort:  
kilosort.run_kilosort:  
kilosort.run_kilosort:  
kilosort.run_kilosort:  
kilosort.run_kilosort:  
kilosort.run_kilosort:  
kilosort.run_kilosort:  
kilosort.run_kilosort:  
kilosort.run_kilosort:  
kilosort.run_kilosort:  
kilosort.run_kilosort:  
kilosort.run_kilosort: Computing preprocessing variables.
kilosort.run_kilosort: Computing preprocessing variables.
kilosort.run_kilosort: Computing preprocessing variables.
kilosort.run_kilosort: Computing preprocessing variables.
kilosort.run_kilosort: Computing preprocessing variables.
kilosort.run_kilosort: Computing preprocessing variables.
kilosort.run_kilosort: Computing preprocessing variables.
kilosort.run_kilos

Error in 210312_GP_JP: 'bool' object has no attribute 'emit'
Error in 210205_GP_JP: Bytes in binary file did not divide evenly, incorrect n_chan_bin ('number of channels' in GUI).


kilosort.run_kilosort: Using GPU for PyTorch computations. Specify `device` to change this.
kilosort.run_kilosort: Using GPU for PyTorch computations. Specify `device` to change this.
kilosort.run_kilosort: Using GPU for PyTorch computations. Specify `device` to change this.
kilosort.run_kilosort: Using GPU for PyTorch computations. Specify `device` to change this.
kilosort.run_kilosort: Using GPU for PyTorch computations. Specify `device` to change this.
kilosort.run_kilosort: Using GPU for PyTorch computations. Specify `device` to change this.
kilosort.run_kilosort: Using GPU for PyTorch computations. Specify `device` to change this.
kilosort.run_kilosort: Using GPU for PyTorch computations. Specify `device` to change this.
kilosort.run_kilosort: Using GPU for PyTorch computations. Specify `device` to change this.
kilosort.run_kilosort: Using GPU for PyTorch computations. Specify `device` to change this.
kilosort.run_kilosort:  
kilosort.run_kilosort:  
kilosort.run_kilosort:  
kilos

Error in 210323_GP_JP: Bytes in binary file did not divide evenly, incorrect n_chan_bin ('number of channels' in GUI).
Error in 210211_GP_JP: Bytes in binary file did not divide evenly, incorrect n_chan_bin ('number of channels' in GUI).


kilosort.run_kilosort: ----------------------------------------
kilosort.run_kilosort: ----------------------------------------
kilosort.run_kilosort: Interpreting binary file as default dtype='int16'. If data was saved in a different format, specify `data_dtype`.
kilosort.run_kilosort: Interpreting binary file as default dtype='int16'. If data was saved in a different format, specify `data_dtype`.
kilosort.run_kilosort: Interpreting binary file as default dtype='int16'. If data was saved in a different format, specify `data_dtype`.
kilosort.run_kilosort: Interpreting binary file as default dtype='int16'. If data was saved in a different format, specify `data_dtype`.
kilosort.run_kilosort: Interpreting binary file as default dtype='int16'. If data was saved in a different format, specify `data_dtype`.
kilosort.run_kilosort: Interpreting binary file as default dtype='int16'. If data was saved in a different format, specify `data_dtype`.
kilosort.run_kilosort: Interpreting binary file as

Error in 210210_GP_JP: Bytes in binary file did not divide evenly, incorrect n_chan_bin ('number of channels' in GUI).
Error in 210317_GP_JP: Bytes in binary file did not divide evenly, incorrect n_chan_bin ('number of channels' in GUI).


kilosort.run_kilosort: Sorting /mnt/prior-data/raw/210413_GP_JP/210413_GP_JP.bin
kilosort.run_kilosort: Sorting /mnt/prior-data/raw/210413_GP_JP/210413_GP_JP.bin
kilosort.run_kilosort: Sorting /mnt/prior-data/raw/210413_GP_JP/210413_GP_JP.bin
kilosort.run_kilosort: Sorting /mnt/prior-data/raw/210413_GP_JP/210413_GP_JP.bin
kilosort.run_kilosort: Sorting /mnt/prior-data/raw/210413_GP_JP/210413_GP_JP.bin
kilosort.run_kilosort: Sorting /mnt/prior-data/raw/210413_GP_JP/210413_GP_JP.bin
kilosort.run_kilosort: Sorting /mnt/prior-data/raw/210413_GP_JP/210413_GP_JP.bin
kilosort.run_kilosort: Sorting /mnt/prior-data/raw/210413_GP_JP/210413_GP_JP.bin
kilosort.run_kilosort: Sorting /mnt/prior-data/raw/210413_GP_JP/210413_GP_JP.bin
kilosort.run_kilosort: Sorting /mnt/prior-data/raw/210413_GP_JP/210413_GP_JP.bin
kilosort.run_kilosort: Sorting /mnt/prior-data/raw/210413_GP_JP/210413_GP_JP.bin
kilosort.run_kilosort: Sorting /mnt/prior-data/raw/210413_GP_JP/210413_GP_JP.bin
kilosort.run_kilosort: -----

Error in 210413_GP_JP: Bytes in binary file did not divide evenly, incorrect n_chan_bin ('number of channels' in GUI).
Error in 210318_GP_JP: filename '/mnt/prior-data/raw/210318_GP_JP/210318_GP_JP.bin' does not exist
Error in 210302_GP_JP: filename '/mnt/prior-data/raw/210302_GP_JP/210302_GP_JP.bin' does not exist
Error in 210217_GP_JP: filename '/mnt/prior-data/raw/210217_GP_JP/210217_GP_JP.bin' does not exist
Error in 210315_GP_JP: filename '/mnt/prior-data/raw/210315_GP_JP/210315_GP_JP.bin' does not exist
Error in 210126_GP_JP: filename '/mnt/prior-data/raw/210126_GP_JP/210126_GP_JP.bin' does not exist
Error in 210311_GP_JP: filename '/mnt/prior-data/raw/210311_GP_JP/210311_GP_JP.bin' does not exist


In [11]:


filename = '/mnt/data/prior-encoding-in-nhp-sc/bin_data/Taz24_Jun25_GPrt_003.bin'

# probe_path = Path('/src/config/s-probe.json')
probe_path = Path('/src/config/new-mapping-s-probe.json')
dtype = np.int32

assert probe_path is not None, 'No probe information exported by SpikeInterface'
probe = io.load_probe(probe_path)

ops, st, clu, tF, Wall, similar_templates, is_ref, est_contam_rate, kept_spikes = run_kilosort(
        settings=settings, probe=probe, filename=filename
        )

kilosort.run_kilosort: Kilosort version 4.0.12
kilosort.run_kilosort: Kilosort version 4.0.12
kilosort.run_kilosort: Kilosort version 4.0.12
kilosort.run_kilosort: Sorting /mnt/data/prior-encoding-in-nhp-sc/bin_data/Taz24_Jun25_GPrt_003.bin
kilosort.run_kilosort: Sorting /mnt/data/prior-encoding-in-nhp-sc/bin_data/Taz24_Jun25_GPrt_003.bin
kilosort.run_kilosort: Sorting /mnt/data/prior-encoding-in-nhp-sc/bin_data/Taz24_Jun25_GPrt_003.bin
kilosort.run_kilosort: ----------------------------------------


kilosort.run_kilosort: ----------------------------------------
kilosort.run_kilosort: ----------------------------------------
kilosort.run_kilosort: Interpreting binary file as default dtype='int16'. If data was saved in a different format, specify `data_dtype`.
kilosort.run_kilosort: Interpreting binary file as default dtype='int16'. If data was saved in a different format, specify `data_dtype`.
kilosort.run_kilosort: Interpreting binary file as default dtype='int16'. If data was saved in a different format, specify `data_dtype`.
kilosort.run_kilosort: Using GPU for PyTorch computations. Specify `device` to change this.
kilosort.run_kilosort: Using GPU for PyTorch computations. Specify `device` to change this.
kilosort.run_kilosort: Using GPU for PyTorch computations. Specify `device` to change this.
kilosort.run_kilosort:  
kilosort.run_kilosort:  
kilosort.run_kilosort:  
kilosort.run_kilosort: Computing preprocessing variables.
kilosort.run_kilosort: Computing preprocessing varia

In [ ]:
kilosort_path = Path('/mnt/data/prior-encoding-in-nhp-sc/bin_data/kilosort4')